In [78]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex, hexcolor_str

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 5: Hydrothermal Venture ---</h2><p>You come across a field of <a href="https://en.wikipedia.org/wiki/Hydrothermal_vent" target="_blank">hydrothermal vents</a> on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.</p>
<p>They tend to form in <em>lines</em>; the submarine helpfully produces a list of nearby <span title="Maybe they're Bresenham vents.">lines of vents</span> (your puzzle input) for you to review. For example:</p>
<pre><code>0,9 -&gt; 5,9
8,0 -&gt; 0,8
9,4 -&gt; 3,4
2,2 -&gt; 2,1
7,0 -&gt; 7,4
6,4 -&gt; 2,0
0,9 -&gt; 2,9
3,4 -&gt; 1,4
0,0 -&gt; 8,8
5,5 -&gt; 8,2
</code></pre>
<p>Each line of vents is given as a line segment in the format <code>x1,y1 -&gt; x2,y2</code> where <code>x1</code>,<code>y1</code> are the coordinates of one end the line segment and <code>x2</code>,<code>y2</code> are the coordinates of the other end. These line segments include the points at both ends. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 1,3</code> covers points <code>1,1</code>, <code>1,2</code>, and <code>1,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,7</code> covers points <code>9,7</code>, <code>8,7</code>, and <code>7,7</code>.</li>
</ul>
<p>For now, <em>only consider horizontal and vertical lines</em>: lines where either <code>x1 = x2</code> or <code>y1 = y2</code>.</p>
<p>So, the horizontal and vertical lines from the above list would produce the following diagram:</p>
<pre><code>.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
</code></pre>
<p>In this diagram, the top left corner is <code>0,0</code> and the bottom right corner is <code>9,9</code>. Each position is shown as <em>the number of lines which cover that point</em> or <code>.</code> if no line covers that point. The top-left pair of <code>1</code>s, for example, comes from <code>2,2 -&gt; 2,1</code>; the very bottom row is formed by the overlapping lines <code>0,9 -&gt; 5,9</code> and <code>0,9 -&gt; 2,9</code>.</p>
<p>To avoid the most dangerous areas, you need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is anywhere in the diagram with a <code>2</code> or larger - a total of <code><em>5</em></code> points.</p>
<p>Consider only horizontal and vertical lines. <em>At how many points do at least two lines overlap?</em></p>
</article>


In [79]:
from pprint import pprint

from more_itertools import minmax


example = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
"""


def counts_points_with_at_least_two_lines_overlap(s: str) -> int:
    lines = [
        [int(i) for i in re.split(r",|\s+->\s+", l)] for l in s.strip().splitlines()
    ]
    points = defaultdict(int)

    for x1, y1, x2, y2 in lines:
        if x1 == x2:
            y_min, y_max = minmax((y1, y2))
            for yy in range(y_min, y_max + 1):
                points[(x1, yy)] += 1
        elif y1 == y2:
            x_min, x_max = minmax((x1, x2))
            for xx in range(x_min, x_max + 1):
                points[(xx, y1)] += 1

    return sum(1 for v in points.values() if v >= 2)


assert counts_points_with_at_least_two_lines_overlap(example) == 5

In [80]:
with open("../input/day5.txt") as f:
    puzzle = f.read()

print(
    f"Part I: {hexcolor_str('#FFD700', str(counts_points_with_at_least_two_lines_overlap(puzzle)))}"
)

Part I: 6564


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>6564</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider <em>diagonal lines</em>.</p>
<p>Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 3,3</code> covers points <code>1,1</code>, <code>2,2</code>, and <code>3,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,9</code> covers points <code>9,7</code>, <code>8,8</code>, and <code>7,9</code>.</li>
</ul>
<p>Considering all lines from the above example would now produce the following diagram:</p>
<pre><code>1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
</code></pre>
<p>You still need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is still anywhere in the diagram with a <code>2</code> or larger - now a total of <code><em>12</em></code> points.</p>
<p>Consider all of the lines. <em>At how many points do at least two lines overlap?</em></p>
</article>

</main>


In [81]:
def counts_points_with_at_least_two_lines_overlap(s: str) -> int:
    lines = [
        [int(i) for i in re.split(r",|\s+->\s+", l)] for l in s.strip().splitlines()
    ]
    points = defaultdict(int)

    for x1, y1, x2, y2 in lines:
        mn, mx = minmax((x1, x2) if x1 != x2 else (y1, y2))

        x_rico = (x2 - x1) // abs(x1 - x2) if x1 != x2 else 0
        y_rico = (y2 - y1) // abs(y1 - y2) if y1 != y2 else 0

        for delta in range(mx - mn + 1):
            points[(x1 + x_rico * delta, y1 + y_rico * delta)] += 1

    return sum(1 for v in points.values() if v >= 2)


assert counts_points_with_at_least_two_lines_overlap(example) == 12

In [82]:
print(f"Part II: { counts_points_with_at_least_two_lines_overlap(puzzle)}")

Part II: 19172


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>19172</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
